In [1]:
import pandas as pd
import numpy as np
import time
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
# import sys
# sys.path.insert(0, 'usr/lib/chromium-browser/chromedriver')

In [3]:
# 댓글 버튼 유무
def is_comment(url_id):
    try:
        driver.find_element_by_xpath(f'//*[@id="Comi{url_id}"]')
        return 1
    except:
        return 0

In [4]:
# 해시태그 갯수
def tag_counts():
    hashtag_cnt = 0
    blogtag_cnt = 0

    try:
        hashtag_cnt = len(driver.find_elements_by_class_name('tag'))

    except Exception as e:
        print(e, 'error1!!')

    try:
        blogtag_cnt = len(driver.find_elements_by_class_name('ell'))

    except Exception as e:
        print(e, 'error2')

    return hashtag_cnt, blogtag_cnt

In [5]:
# 블로그 content
def contents_cralwing():
    content = 'error'
    # 기존 블로그
    try:
        content = driver.find_element_by_class_name("se-main-container")
        time.sleep(1.5)
        # print(content)
        # print(current_url)
        return content
    except:
        pass
    
    # 예전 블로그
    try:
        content = driver.find_element_by_id("postViewArea")
        return content
    except:
        pass

    return content

In [6]:
options = webdriver.ChromeOptions()
# option.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome('./chromedriver', options=options)

In [ ]:
#'블루투스' 반드시 포함 코드
"""
url = f'https://search.naver.com/search.naver? \
        date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}\
        +%2B블루투스&sm=tab_pge&srchby=all&st=sim&where=post&start={page * 10 + 1}'
"""

In [7]:
lst_keyword = ['블루투스 소정의 받았습니다'] ##### keyword 변경 #####

Data = pd.DataFrame()

for keyword in lst_keyword:
    for page in range(1, 11):
        print(f'======================================{page}=====================================')
        try:
            url = f'https://search.naver.com/search.naver? \
                    date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}\
                    &sm=tab_pge&srchby=all&st=sim&where=post&start={page * 10 + 1}'
            driver.get(url)
            time.sleep(3.5)

            lst_title = []
            lst_url = []
            lst_content = []
            lst_comment = []
            lst_image = []
            lst_video = []
            lst_link = []
            lst_hashTag = []
            lst_blogTag = []
            
            df = pd.DataFrame()

            for i in range(1, 11):
                try:
                    title = driver.find_element_by_xpath(f'//*[@id="sp_blog_{i}"]/dl/dt/a').text
                    print(f'{i}: title: ', title)

                    # 타이틀 클릭
                    driver.find_element_by_xpath(f'//*[@id="sp_blog_{i}"]/dl/dt/a').click()
                    time.sleep(2.5)

                    # window 탭 이동
                    driver.switch_to.window(driver.window_handles[1])
                    time.sleep(2.5)

                    # 프레임 링크 전환
                    driver.switch_to.frame("mainFrame")
                    time.sleep(1.5)
                    
                    # 블로그 content
                    current_url = driver.current_url
                    url_id = current_url.split('/')[-1]
                    content = contents_cralwing()
                    if content == 'error': raise Exception('contents error')
                        
                    # 댓글 버튼의 유무
                    comment = is_comment(url_id)
                    time.sleep(1.0)
                    # print(comment)
                    
                    # 태그 갯수
                    hashTags, blogTags = tag_counts()
                    time.sleep(1.0)
                    
                    try: 
                        # 이미지 갯수
                        image = len(driver.find_elements_by_class_name('se-image-resource'))
                        time.sleep(1.0)
                        # print(image)
                        
                        # 비디오 갯수
                        video = len(driver.find_elements_by_class_name('u_rmcplayer_control_bg'))
                        time.sleep(1.0)
                        # print(video)
                        
                        # 링크 갯수
                        link1 = len(driver.find_elements_by_class_name("se-link"))  
                        link2 = len(driver.find_elements_by_class_name("se-oglink-url"))
                        links = (link1 + link2)
                        # print(links)
                        
                        lst_image.append(image)
                        lst_video.append(video)
                        lst_link.append(links)
                        
                    except Exception as e:
                        print('Len error: ', e)
                        lst_image.append(0)
                        lst_video.append(0)
                        lst_link.append(0)
                        pass

                    lst_title.append(title)
                    lst_url.append(current_url)
                    lst_content.append(content.text)
                    lst_comment.append(comment)
                    lst_hashTag.append(hashTags)
                    lst_blogTag.append(blogTags)

                    df = pd.DataFrame({'title': lst_title, 'content': lst_content, 'comment': lst_comment,
                                       'image_count': lst_image, 'video_count': lst_video, 'link_count': lst_video,
                                       'hash_tag_count': lst_hashTag, 'blog_tag_count': lst_blogTag, 'url': lst_url})
                    df.to_csv(f'./crawling/IT_crawling_{keyword}_page{page}.csv')
                    # print(df.info())
                     
                except Exception as e:
                    print('error1: ', e)
                    continue
                
                finally:
                    print('.')
                    driver.close()
                    time.sleep(2.0)
                    driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(e)
            continue
        
        finally:
            print('> 끝!')
            print(df.info())
            Data = pd.concat([Data, df], ignore_index=True)
            Data.to_csv(f'./crawling/IT_crawling_{keyword}.csv')
        
driver.close()

======================================1=====================================
1: title:  무선충전기가 블루투스 스피커와 탁상시계까지 된다고!!
.
2: title:  휴대용 블루투스 스피커 소노로 이지
.
3: title:  가성비 블루투스 이어폰 추천 QCY T1 TWS
.
4: title:  JBL 블루투스이어폰 가성비 좋고 음질 좋은 JBL tune120
.
5: title:  블루투스 키보드, 윈도우 애플 안드로이드 상관없는...
.
6: title:  블루투스 마우스 로지텍 MX Master 3 업무용 추천 이유
.
7: title:  무선 블루투스 헤드폰, 하만 신제품 런칭 행사를 가다!
.
8: title:  블루투스 스피커 품은 올인원 오디오 소노로 라운지
.
9: title:  노이즈캔슬링 블루투스 이어폰 젠하이저 모멘텀 트루...
.
10: title:  블루투스 키보드 고민? 재주 많은 로지텍 K380을 살펴보자
error1:  Message: mainFrame

.
> 끝!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      9 non-null      object
 1   content    9 non-null      object
 2   comment    9 non-null      int64 
 3   image_cnt  9 non-null      int64 
 4   video_cnt  9 non-null      int64 
 5   link_cnt   9 non-null      int64 
 6   hash_tag   2 non-null      object
 7 

1: title:  야외용으로도 좋은 블루투스 스피커 LG 포터블 PH3
error1:  contents error
.
2: title:  LG 포터블 블루투스 스피커로 작업실분위기 up ♪
.
3: title:  스타일리시한 블루투스 이어폰, LG 톤플러스 HBS-900
.
4: title:  무선 블루투스 스피커 추천 소니 SRS-XB21 전천후...
error1:  contents error
.
5: title:  소니의 역습! 노이즈캔슬링 블루투스 헤드폰 MDR-1000X 개봉기
error1:  contents error
.
6: title:  LG 블루투스 헤드셋 톤플러스 웨어러블 기기 추천
.
7: title:  LG 롤리키보드, 접이식 블루투스 키보드라니 아이디어 참 좋네
.
8: title:  디자인과 사운드 두 마리 토끼를 동시에 잡은 블루투스...
error1:  contents error
.
9: title:  LG 블루투스 프로젝터 미니빔TV PW700 개봉기! - 캠핑 휴대용...
error1:  Message: mainFrame

.
10: title:  블루투스 이어폰 톤플러스 스튜디오 HBS-W120, 톤플러스...
error1:  Message: mainFrame

.
> 끝!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      4 non-null      object 
 1   content    4 non-null      object 
 2   comment    4 non-null      int64  
 3   image_cnt  4 non-null      int64  
 4   video_cnt  4 non-null     

In [9]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      57 non-null     object
 1   content    57 non-null     object
 2   comment    57 non-null     int64 
 3   image_cnt  57 non-null     int64 
 4   video_cnt  57 non-null     int64 
 5   link_cnt   57 non-null     int64 
 6   hash_tag   4 non-null      object
 7   blog_tag   56 non-null     object
 8   url        57 non-null     object
dtypes: int64(4), object(5)
memory usage: 4.1+ KB
